In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [133]:
import pandas as pd

mags = pd.read_csv("C:/Users/juniu/PycharmProjects/SubFinda_gradio/final_mags.csv")
mags["row_number"] = mags["row_number"].astype("Int64")
mags = mags.dropna(subset=["row_number"])  # Drop rows with missing row_number
mags["row_number"] = mags["row_number"].astype(int)

In [151]:
import csv
import re

mags["tagged_description"] = mags["row_number"].astype(str) + " " + mags["description"].fillna("")
mags["tagged_description"].to_csv("tagged_description.txt", index=False, header=False)


In [152]:
mags["tagged_description"]

0     1 McSweeney?s is a nonprofit publishing compan...
1     2 At The Yale Review, we believe in the power ...
2     3 This magazine aims to merge the worlds of Ar...
3     4 Taco Bell Quarterly seeks literary/creative ...
4     5 We welcome all types of poetry (including, b...
5     6 The Acentos Review shoots from the spring th...
6     7 Aethlon: The Journal of Sport Literature is ...
7     8 DIAGRAM is an online literary magazine found...
8     9 We accept everything. Text-based. Visual. Te...
9     10 We are on the lookout for contemporary lite...
10    11 Action, Spectacle welcomes submissions via ...
11    12 We appreciate good writing in any genre. We...
12    13 ?Agapanthus? Amaryllidaceae agapanthus From...
13    14 Our literary journal, ionosphere, focuses o...
14    15 Simply put, Across the Margin is an online ...
15    16 We take our name from the dotted line on a ...
16    17 One day in the summer of 2013, three friend...
17    19 AURORE is real, non-fiction erotica, wr

In [153]:
print(mags[["row_number", "tagged_description"]].head(10))

   row_number                                 tagged_description
0           1  1 McSweeney?s is a nonprofit publishing compan...
1           2  2 At The Yale Review, we believe in the power ...
2           3  3 This magazine aims to merge the worlds of Ar...
3           4  4 Taco Bell Quarterly seeks literary/creative ...
4           5  5 We welcome all types of poetry (including, b...
5           6  6 The Acentos Review shoots from the spring th...
6           7  7 Aethlon: The Journal of Sport Literature is ...
7           8  8 DIAGRAM is an online literary magazine found...
8           9  9 We accept everything. Text-based. Visual. Te...
9          10  10 We are on the lookout for contemporary lite...


In [154]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 718, which is longer than the specified 0
Created a chunk of size 1480, which is longer than the specified 0
Created a chunk of size 783, which is longer than the specified 0
Created a chunk of size 848, which is longer than the specified 0
Created a chunk of size 263, which is longer than the specified 0
Created a chunk of size 1051, which is longer than the specified 0
Created a chunk of size 334, which is longer than the specified 0
Created a chunk of size 652, which is longer than the specified 0
Created a chunk of size 132, which is longer than the specified 0
Created a chunk of size 610, which is longer than the specified 0
Created a chunk of size 543, which is longer than the specified 0
Created a chunk of size 996, which is longer than the specified 0
Created a chunk of size 846, which is longer than the specified 0
Created a chunk of size 367, which is longer than the specified 0
Created a chunk of size 314, which is longer than the specified 0
Created 

In [155]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"1 McSweeney?s is a nonprofit publishing company based in San Francisco. As well as operating a daily humor website, we also publish Timothy McSweeney?s Quarterly Concern, Illustoria Magazine, and an ever-growing selection of books under various imprints. You can buy all of these things from our online store. You can support us today by making a donation.McSweeney?s exists to champion ambitious and inspired new writing, and to challenge conventional expectations about where it?s found, how it looks, and who participates.We?re here to discover things we love, help them find their most resplendent form, and imagine new ways to bring them to you, Dennis. (If you are not Dennis, kindly ignore the aforementioned."')

In [160]:
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

embedding = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
db_mags = Chroma.from_documents(documents, embedding=embedding)


In [161]:
from openai import OpenAI
from langchain_chroma import Chroma
from langchain_core.embeddings import Embeddings
import numpy as np

# Your OpenAI client with project ID
client = OpenAI(
    api_key="",
    project=""
)

# Custom wrapper (simplified)
class MyOpenAIEmbedding(Embeddings):
    def embed_documents(self, texts):
        return [client.embeddings.create(input=t, model="text-embedding-3-small").data[0].embedding for t in texts]

    def embed_query(self, text):
        return client.embeddings.create(input=text, model="text-embedding-3-small").data[0].embedding

# Then pass this into Chroma
embedding = MyOpenAIEmbedding()
db = Chroma.from_documents(documents, embedding=embedding)


In [158]:
query = ("a magazine to submit horror focused work to")
docs = db.similarity_search(query, k = 16)
docs

[Document(id='4d9e583f-8d3d-499c-b8f9-f87b0f01ca35', metadata={'source': 'tagged_description.txt'}, page_content='45As an online press for dark literature, we publish our nonprofit magazine issues annually on All Hallows? Eve (Oct 31st). We also accept submissions for publication on our website under Dark Reads. There?s no theme for these submissions, however we are always interested in horror and dark fiction surrounding the thinning of the veil. Give us ghosts, ghouls, and small town cryptids. Unleash something ancient from the abandoned crypt and make us wish you hadn?t.'),
 Document(id='b4b3d83e-5e3e-49ff-ac59-9e3d88e9ad9b', metadata={'source': 'tagged_description.txt'}, page_content='"0045As an online press for dark literature, we publish our nonprofit magazine issues annually on All Hallows? Eve (Oct 31st). We also accept submissions for publication on our website under Dark Reads. There?s no theme for these submissions, however we are always interested in horror and dark fiction

In [159]:
def retrieve_semantic_recommendations(query: str, top_k: int = 16) -> pd.DataFrame:
    docs = db.similarity_search(query, k=top_k)
    row_ids = []

    for doc in docs:
        match = re.match(r'^(\d+)', doc.page_content.strip())
        if match:
            row_ids.append(int(match.group(1)))

    # Maintain the ranking order by reconstructing the DataFrame
    ranked_matches = [mags[mags["row_number"] == rid] for rid in row_ids]
    return pd.concat(ranked_matches).head(top_k)


In [140]:
retrieve_semantic_recommendations("a magazine to submit horro focused work to")

,row_number,market_name,genres,response_time,url of thumbnail,acceptance rate,followers,founded,country,description,submission_guidelines,tagged_description
44,45,Thin Veil Press,"Fiction, Poetry",56 days,https://www.chillsubs.com/_next/image?url=http...,20.97%,100,2023.0,United Kingdom,"As an online press for dark literature, we pub...",https://thinveilpress.co.uk/submit,"45 As an online press for dark literature, we ..."
36,37,Pseudopod,Fiction,58 days,https://cdn.duotrope.com/covers/1276-250303221...,2.65%,"6,000",2006.0,united States,"For over a decade, PseudoPod has been bringing...",https://pseudopod.org/submissions/,"37 For over a decade, PseudoPod has been bring..."
2,3,Suburban Witchcraft Magazine,"Fiction, Nonfiction, Poetry, Hybrid, Multimedia",11 days,https://www.chillsubs.com/_next/image?url=http...,96.72%,"1,000",2022.0,Serbia,This magazine aims to merge the worlds of Art ...,https://suburbanwitchcraftmagazine.wordpress.c...,3 This magazine aims to merge the worlds of Ar...
13,14,Ionosphere,"Nonfiction, Poettry",76 days,https://www.chillsubs.com/_next/image?url=http...,72.73%,200,2023.0,United kingdom,"Our literary journal, ionosphere, focuses on t...",https://ampliconpress.com/submit-to-ionosphere,"14 Our literary journal, ionosphere, focuses o..."
28,29,Snowflake Magazine,"Nonfiction, Hybrid, Multimedia, Fiction, Poetry",40 days,https://www.chillsubs.com/_next/image?url=http...,80.00%,"2,000",2021.0,United Kingdom,Snowflake magazine is looking for submissions ...,https://www.snowflakeculture.com/submit,29 Snowflake magazine is looking for submissio...
35,36,Apex Magazine,Fiction,34 days,https://www.chillsubs.com/_next/image?url=http...,0.81%,"35,000",2005.0,United States,Apex Magazine is an online zine of fantastical...,https://www.apexbookcompany.com/a/blog/apex-ma...,36 Apex Magazine is an online zine of fantasti...
29,30,Humana Obscura,"Poetry, Multimedia, Fiction",58 days,https://cdn.duotrope.com/covers/31086-25060604...,15.58%,"4,000",2020.0,United States,Humana Obscura is an independent literary maga...,https://www.humanaobscura.com/submit,30 Humana Obscura is an independent literary m...
